# Этап L4

In [1]:
!pip install lightfm

In [2]:
pip install lightgbm

In [ ]:
import pandas as pd
import numpy as np
from numpy import save, load
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from scipy import sparse
from scipy.sparse import csr_matrix, coo_matrix
import itertools
import random
from tqdm import tqdm
from lightfm import LightFM
from lightfm.evaluation import recall_at_k, precision_at_k, auc_score
import lightgbm

In [ ]:
path = "/content/drive/MyDrive/WB School/data.csv.gzip"
df = pd.read_csv(path, compression="gzip")
df["order_ts"] = pd.to_datetime(df["order_ts"])

# Создание классов

## DataPreparation

Класс **DataPreparation** позволяет выполнить все необходимые преобразования над данными для дальнейшего обучения моделей.

Метод extract_reluctant_users исключит из рассмотрения тех пользователей, которые взаимодействовали с товарами слишком малое число раз. Метод drop_rare_items служит для аналогичной цели с той разницей, что отбрасывает слишком редко заказываемые товары.

Метод train_test поделит выборку на обучающую и тестовую части либо по времени, либо по пропорции между выборками.

Метод common_only оставит только те взаимодействия, которые: (1) осуществлены пользователями, сделавшими заказ и в обучающей, и в тестовой частяъ выборки; (2) связаны с товарами, заказанными и в обучающей, и в тестовой выборках. С рациональной точки зрения, оценить качество модели по пользователю, принадлежащему лишь одной выборке, не получится, поэтому их можно отбросить. С технической точки зрения, это снизит вычислительную сложность.

Метод csr_matrix_via_encoder построит из обучающей и тестовой выборок 2 разрезженные матрицы csr-формата, причём пользователи и товары получат одинаковую индексацию в обеих новых матрицах.

In [ ]:
class DataPreparation:

  def __init__(self):
    pass


  def extract_reluctant_users(self, data, threshold=5, both=False):
    """
    Метод отбрасывает/отделяет пользователей, которые сделали заказы в количестве, меньшем порогового значения.

    Параметры:
    ----------
    data: Dataframe
      pandas-датафрейм с историей взаимодействий пользователей с товарами формата исходного датафрейма.
    threshold: int
      пороговое целочисленное значение минимального количества заказов у пользователя, достаточное для того, чтобы не быть отброшенным/отделённым.

    На выход:
    ----------
    data: Dataframe
      pandas-датафрейм, все пользователи которого сделали по меньшей мере заказов в количестве, не меньшем порогового значения.
    data_reluctants: Dataframe
      Если указано both=True, то недостаточно активные пользователи будут не отброшены, а сохранены в отдельный pandas-датафрейм.
    """
    data = data.drop_duplicates()
    data_count = data.groupby(["user_id", "item_id"], as_index=False).count().rename(columns={"order_ts": "counter"})
    data_count_users = data_count.groupby("user_id", as_index=False)["counter"].sum()
    users = data_count_users.loc[data_count_users.counter <= threshold, "user_id"].values
    data = data[~data.user_id.isin(users)]
    if both == True:
      data_reluctants = data[data.user_id.isin(users)]
      return data_reluctants, data
    else:
      return data


  def drop_rare_items(self, data, threshold=2):
    """
    Метод отбрасывает товары, с которыми пользователи взаимодействовали меньше, чем threshold раз.

    Параметры:
    ----------
    data: Dataframe
      pandas-датафрейм с историей взаимодействий пользователей с товарами формата исходного датафрейма.
    threshold: int
      пороговое целочисленное значение минимального количества заказов товара, достаточное для того, чтобы товар ге был отброшен.

    На выход:
    ----------
    data: Dataframe
       pandas-датафрейм, в котором остались взаимодействия исключительно с достаточно популярными товарами, т.е. с товарами, которые были заказаны более, чем threshold раз.
    """
    data_temp = data.drop_duplicates()
    data_count = data_temp.groupby(["user_id", "item_id"], as_index=False).count().rename(columns={"order_ts": "counter"})
    data_count_items = data_count.groupby("item_id", as_index=False)["counter"].sum()
    items = data_count_items.loc[data_count_items.counter <= threshold, "item_id"].values
    data = data[~data.item_id.isin(items)]
    return data


  def train_test(self, data, by="time", test_weeks=1, test_size=0.2):
    """
    Метод делит выборку на обучающую и тестовую одним из двух способов: по времени, что необходимо для получения глобальных обучающей и тестовой выборок, и по доле, что используется для разделения на локальные обучающую и тестовую выборки.

    Параметры:
    ----------
    data: Dataframe
      pandas-датафрейм с историей взаимодействий пользователей с товарами формата исходного датафрейма.
    by: str
      Строка, принимающая 2 значения. Если необходимо разделить по времени, то by='time'. Если же по долям, то by='percents'.
    test_weeks: float
      Действительное ненулевое число, обозначающее, сколько недель необходимо отнести в тестовую выборку, если указано by='time' и разбиение необходимо сделать по времени.
    test_size: float
      Действительное число, отражающее проворцию между размером тестовой выборки и размером исходной выборки, если указано by='percents'.

    На выход:
    ----------
    train: Dataframe
      Обучающая выборка.
    test: Dataframe
      Тестовая выборка.
    """
    if by == "time":
      n_folds = 13 / test_weeks
      delta = (data["order_ts"].max() - data["order_ts"].min()) / n_folds
      edge = data["order_ts"].max() - delta
      train = data.loc[data["order_ts"] <= edge]
      test = data.loc[data["order_ts"] > edge]
    elif by == "percents":
      train_size = 1 - test_size
      idx = int(len(data) * train_size)
      train = data[:idx]
      test = data[idx:]
    return train, test


  def common_only(self, data_first, data_second, both=True, common_col="user_id"):
    """
    Метод оставляет в обоих датафреймах только те наблюдения, которые принадлежат пересечению по колонке column.

    Параметры:
    ----------
    data_first: Dataframe
      Первый pandas-датафрейм с историей взаимодействий пользователей с товарами формата исходного датафрейма.
    data_second: Dataframe
      Второй pandas-датафрейм с историей взаимодействий пользователей с товарами формата исходного датафрейма.
    column: str
      Название колонки, по которой надо исхать общие значения. Принимает 2 значения: 'user_id' или 'item_id'.


    На выход:
    ----------
    data_first: Dataframe
      Первый датафрейм, в котором остались только те наблюдения, которые имеют общие значения по колонке column со вторым датафреймом data_second.
    data_second: Dataframe
      Первый датафрейм, в котором остались только те наблюдения, которые имеют общие значения по колонке column с первым датафреймом data_first.
    """
    if both == True:

      column = "user_id"
      common = list(set(data_first[column]).intersection(set(data_second[column])))
      data_first = data_first[data_first[column].isin(common)]
      data_second = data_second[data_second[column].isin(common)]

      column = "item_id"
      common = list(set(data_first[column]).intersection(set(data_second[column])))
      data_first = data_first[data_first[column].isin(common)]
      data_second = data_second[data_second[column].isin(common)]

      column = "user_id"
      common = list(set(data_first[column]).intersection(set(data_second[column])))
      data_first = data_first[data_first[column].isin(common)]
      data_second = data_second[data_second[column].isin(common)]

      column = "item_id"
      common = list(set(data_first[column]).intersection(set(data_second[column])))
      data_first = data_first[data_first[column].isin(common)]
      data_second = data_second[data_second[column].isin(common)]

    elif both == False:

      common = list(set(data_first[common_col]).intersection(set(data_second[common_col])))
      data_first = data_first[data_first[common_col].isin(common)]
      data_second = data_second[data_second[common_col].isin(common)]

    return data_first, data_second


  def csr_matrix_via_encoder(self, data_first, data_second):
    """
    Метод преобразует два pandas-датафрейма в 2 разрезженные csr-матрицы с одинаковым порядком индексов пользователей и айтемов.

    Параметры:
    ----------
    data_first: Dataframe
      Первый pandas-датафрейм с историей взаимодействий пользователей с товарами формата исходного датафрейма.
    data_second: Dataframe
      Второй pandas-датафрейм с историей взаимодействий пользователей с товарами формата исходного датафрейма.

    На выход:
    ----------
    data_first_sparse: csr-matrix
      Разрезженная csr-матрица, соответствующая первому датафрейму data_first.
    data_second_sparse: csr-matrix
      Разрезженная csr-матрица, соответствующая второму датафрейму data_second.
    data_first: Dataframe
      Датафрейм, отличающийся от датафрейма data_first, поступившего на вход, наличием 2 колонок с новой индексацией для user_id и item_id, соответствующей отображению в csr-матрицу.
    data_second: Dataframe
      Датафрейм, отличающийся от датафрейма data_first, поступившего на вход, наличием 2 колонок с новой индексацией для user_id и item_id, соответствующей отображению в csr-матрицу.
    """
    data_first = data_first.groupby(["user_id", "item_id"], as_index=False).count().rename(columns={"order_ts": "counter"})
    data_second = data_second.groupby(["user_id", "item_id"], as_index=False).count().rename(columns={"order_ts": "counter"})

    data_first = data_first.sort_values("user_id")
    data_second = data_second.sort_values("user_id")

    user_encoder, item_encoder = LabelEncoder(), LabelEncoder()

    users_final = set(data_first.user_id.unique()).intersection(set(data_second.user_id.unique()))
    user_encoder.fit(list(users_final))

    all_items = set(data_first.item_id.unique()).union(set(data_second.item_id.unique()))
    item_encoder.fit(list(all_items))

    data_first["user_new_id"] = user_encoder.transform(data_first["user_id"])
    data_second["user_new_id"] = user_encoder.transform(data_second["user_id"])

    data_first["item_new_id"] = item_encoder.transform(data_first["item_id"])
    data_second["item_new_id"] = item_encoder.transform(data_second["item_id"])

    matrix_shape = len(user_encoder.classes_), len(item_encoder.classes_)

    data_first_sparse = coo_matrix((list(data_first.counter.astype(np.float32)),
                                   (list(data_first.user_new_id.astype(np.int64)),
                                    list(data_first.item_new_id.astype(np.int64)))), shape=matrix_shape)
    data_first_sparse = data_first_sparse.tocsr()

    data_second_sparse = coo_matrix((list(data_second.counter.astype(np.float32)),
                                    (list(data_second.user_new_id.astype(np.int64)),
                                     list(data_second.item_new_id.astype(np.int64)))), shape=matrix_shape)
    data_second_sparse = data_second_sparse.tocsr()

    users = sorted(list(set(coo_matrix(data_first_sparse).row)))
    items = sorted(list(set(coo_matrix(data_first_sparse).col)))

    return data_first_sparse, data_second_sparse, data_first, data_second, users, items


  def users_items(self, data_sparse):
    """
    Метод извлекает user_id и item_id, для которых понадобится вычислять скоры.

    Параметры:
    ----------
    data_sparse: csr_matrix
      Разрезженная csr-матрица. В контексте задачи скоры необходимо считать по обучающей выборке, поэтому это матрица, соответствующая ей.
    """
    users = sorted(list(set(coo_matrix(data_sparse).row)))
    items = sorted(list(set(coo_matrix(data_sparse).col)))
    return users, items

## CandidatesExtractor

Класс **CandidatesExtractor** отвечает за отбор кандидатов в ранжирование. Помимо идентификаторов товаро-кандидатов, на выход получим соответствующие им скоры и ранги, которые и будут фичами для градиентного бустинга, использующегося для ранжирования.

Метод fit отвечает за обучение моделей из библиотеки [LightFM](https://making.lyst.com/lightfm/docs/home.html):

*   Weighted Approximate-Rank Pairwise loss или [WARP](https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/37180.pdf);
*   Bayesian Personalized Ranking или [BPR](https://arxiv.org/ftp/arxiv/papers/1205/1205.2618.pdf);
*   Logistic Matrix Factorization или [LMF](https://web.stanford.edu/~rezab/nips2014workshop/submits/logmat.pdf);
*   k-Order Statistic Weighted Approximate-Rank Pairwise loss или [k-OS WARP](https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/41534.pdf).

Опробованные модели из библиотеки [implicit](https://github.com/benfred/implicit) показали себя хуже. Качество моделей по метрике Recall@20 среди моделей LightFM разнится несильно, поэтому в методе можно обучить все 4 модели. При этом из-за технических ограничений в RAM для отбора кандидатов следует использовать лишь 2 из них.

Метод evaluate используется для оценки модели по 3 метрикам качества: ROC-AUC, Precision@K и Recall@K.

В библиотеке LightFM не предусмотрено структурированное извлечение скоров айтемов. Это позволяет сделать метод calculate_scores. При том, что он работает довольно долго, RAM будет загружаться незначительно.

Метод candidates_extraction отбирает кандидаты. Для воспроизведения результатов на использованных ранее данных можно не запускать вычисления заново, а загрузить и работать с предпосчитанными скорами товаров.

In [ ]:
class CandidatesExtractor:

  def __init__(self):
    pass


  def fit(self, train_data_csr, loss, parameters, random_state=42):
    """
    Метод обучит модели первого уровня с оптимальными значениями параметров, подобранными заранее на основании кросс-валидации.

    Параметры:
    ----------
    train_data_csr: csr-matrix
      Разрезженная csr-матрица, полученная из обучающей выборки.
    loss: str
      Функция потерь, отражающая, какого типа модель необходимо обучить. Принимает значения 'warp', 'bpr', 'logistic' и 'warp-kos'.
    parameters: dict
      Набор параметров для обучения модели.

    На выход:
    ----------
    model: lightfm.LightFM
      Обученная LightFM-модель с параметрами parameters.
    """
    if loss == "warp":
      model = LightFM(no_components=parameters["no_components"],
                      learning_schedule=parameters["learning_schedule"],
                      loss=loss,
                      learning_rate=parameters["learning_rate"],
                      item_alpha=parameters["item_alpha"],
                      user_alpha=parameters["user_alpha"],
                      max_sampled=parameters["max_sampled"],
                      random_state=random_state)
    elif loss == "bpr":
      model = LightFM(no_components=parameters["no_components"],
                      learning_schedule=parameters["learning_schedule"],
                      loss=loss,
                      learning_rate=parameters["learning_rate"],
                      item_alpha=parameters["item_alpha"],
                      user_alpha=parameters["user_alpha"],
                      random_state=random_state)
    elif loss == "logistic":
      model = LightFM(no_components=parameters["no_components"],
                      learning_schedule=parameters["learning_schedule"],
                      loss=loss,
                      learning_rate=parameters["learning_rate"],
                      item_alpha=parameters["item_alpha"],
                      user_alpha=parameters["user_alpha"],
                      random_state=random_state)
    elif loss == "warp-kos":
      model = LightFM(no_components=parameters["no_components"],
                      k=parameters["k"],
                      n=parameters["n"],
                      learning_schedule=parameters["learning_schedule"],
                      loss=loss,
                      learning_rate=parameters["learning_rate"],
                      item_alpha=parameters["item_alpha"],
                      user_alpha=parameters["user_alpha"],
                      max_sampled=parameters["max_sampled"],
                      random_state=random_state)
    else:
      print("В библиотеке LightFM такой модели нет. Параметр loss может принимать 1 из 4 значений: 'warp', 'bpr', 'logistic' или 'warp-kos'.")

    model.fit(train_data_csr, epochs=parameters["epochs"])
    return model


  def evaluate(self, model, test_data_csr, metric="recall", K_items=20):
    """
    Метод оценит качество модели по указанной метрике.

    Параметры:
    ----------
    model: lightfm.LightFM
      Обученная модель из библиотеки LightFM.
    test_data_csr: csr-matrix
      Разрезженная csr-матрица, полученная из тестовой выборки.
    metric: str
      Метрика качества модели, по которой будет оцениваться модель. В библиотеке LightFM доступно 4 метрики качества: 'auc_score', 'precision_at_k', 'recall_at_k' и 'reciprocal_rank', однако последний принимать во внимание не будем.
    K_items: int
      Сколько айтемов учитывать при вычислении метрики качества. По умолчанию равно 20.

    На выход:
    ----------
    metric_score: float
      Значение указанной метрики качества.
    """
    if metric == "recall":
      metric_score = recall_at_k(model=model,
                                 test_interactions=test_data_csr,
                                 k=K_items).mean()
    elif metric == "precision":
      metric_score = precisiob_at_k(model=model,
                                    test_interactions=test_data_csr,
                                    k=K_items).mean()
    elif metric == "auc_score":
      metric_score = auc_score(model=model,
                               test_interactions=test_data_csr,
                               k=K_items).mean()
    else:
      print("Такой метрики нет или она (пока) не учтена. Параметр metric может принимать 1 из 3 значений: 'auc_score', 'precision_at_k', 'recall_at_k', причём для модели отбора кандидатов самым важным является 'recall_at_k'.")

    return metric_score


  def calculate_scores(self, model, users, items, items_number=50):
    """
    Вычисление скоров айтемов сразу для всех пользователей является технически тяжёлым и требует много RAM при большом числе юзеров и/или айтемов. Метод calculate_scores() вычислит скоры без сильной нагрузки на RAM, но
    для этого потребуется время. Функция применяется, когда необходимо посчитать скоры, т.к. нет предрассчитанных скоров.

    Параметры:
    model: lightfm.LightFM
      Предварительно обученная модель из библиотеки LightFM.
    users: list
      Пользователи, для которых будем считать скоры айтемов и отбирать кандидатов.
    items: list
      Товары, среди которых будем отбирать товары-кандидаты в ранжирование для второго этапа.
    items_number: int
      Количество товаров-кандидатов от модели model. Если взять значение больше 50, то вычисления могут не поместиться в оперативную память, поэтому следует брать 50 айтемов.

    На выход:
    ----------
    pairs: list
      Множество пар вида (айтем, скор) для каждого пользователя из users.
    """
    user_biases = model.user_biases[users]
    item_biases = model.item_biases[items]
    item_embeddings = model.item_embeddings[items]
    user_embeddings = model.user_embeddings[users]

    first_N_scores = user_embeddings.dot(item_embeddings[:items_number].T) + user_biases.reshape(-1,1) + item_biases[:items_number].reshape(1,-1)

    pairs = list()

    # Нумеруем первые N айтемов, чтобы не потеряться в нумерации, ведь она не совпадает с исходными item_id
    for i in range(len(first_N_scores)):
      user_scores = list()
      for elem in enumerate(first_N_scores[i]):
        user_scores.append(elem)
      pairs.append(user_scores)

    # Отбираем N (=items_number) айтемов с наибольшим скором, которые и будут кандидатами от модели model
    for u in tqdm(range(len(user_embeddings))):
      for i in range(items_number, len(item_embeddings)):
        score = list(user_embeddings[u:(u + 1)].dot(item_embeddings[i:(i+1)].T) + user_biases[:1].reshape(-1,1) + item_biases[i:(i+1)].reshape(1,-1))[0][0]
        pair = (i, score)
        pairs[u].append(pair)
        pairs[u] = sorted(pairs[u], key=lambda x: x[-1], reverse=True)
        pairs[u].remove(pairs[u][-1])

    return pairs


  def candidates_extraction(self, model_type, path, prefix, users, items, model=None, top=50, items_number=50, precomputed_scores=True):
    """
    Метод отбирает кандидатов от модели, которые пойдут на вход в итоговое ранжирование.

    Параметры:
    ----------
    model: lightfm.LightFM
      Предварительно обученная модель из библиотеки LightFM.
    model_type: str
      Название модели, их которой извлекаются кандидаты. Необходимо для загрузки предпосчитанных скоров. Принимает значения 'warp', 'bpr', 'lmf' и 'warp_kos'.
    path: str
      Путь к скорам товаров-кандидатов от модели для каждого юзера из users. Необходимо для загрузки предпосчитанных скоров.
    prefix: str
      Для разных выборок из юзеров могут быть разные названия файлов со скорами, поэтому параметр позволит хранить все эмбеддинги.
    users: list
      Пользователи, для которых будем считать скоры айтемов и отбирать кандидатов. Потребуется, если скоры предварительно не вычислены.
    items: list
      Товары, среди которых будем отбирать товары-кандидаты в ранжирование для второго этапа. Потребуется, если скоры предварительно не вычислены.
    top: int
      Сколько кандидатов от модели необходимо извлечь. Болшое значение может привести к тому, что вычисления не поместятся в RAM.
    items_number: int
      Количество товаров-кандидатов от модели model. Если взять значение больше 50, то вычисления могут не поместиться в оперативную память, поэтому следует брать 50 айтемов. Тоже потребуется только для вычисления скоров.

    На выход:
    ----------
    model_pairs: numpy ndarray
      Множество пар вида (товар, скор) для каждого пользователя из users.
    model_dict: dict
      Словарь, ключами которого явлюятся id пользователей, соответствующими им значениями - id товаров-кандидатов от модели.
    """
    if precomputed_scores == True:
      path_pairs = path + model_type + "_pairs_" + prefix + ".npy"
      pairs = load(path_pairs)
    elif precomputed_scores == False:

      user_biases = model.user_biases[users],
      item_biases = model.item_biases[items]
      item_embeddings = model.item_embeddings[items]
      user_embeddings = model.user_embeddings[users]

      first_N_scores = user_embeddings.dot(item_embeddings[:items_number].T) + user_biases.reshape(-1,1) + item_biases[:items_number].reshape(1,-1)

      pairs = list()

      # Нумеруем первые N айтемов, чтобы не потеряться в нумерации, ведь она не совпадает с исходными item_id
      for i in range(len(first_N_scores)):
        user_scores = list()
        for elem in enumerate(first_N_scores[i]):
          user_scores.append(elem)
        pairs.append(user_scores)

      # Отбираем N (=items_number) айтемов с наибольшим скором, которые и будут кандидатами от модели model
      for u in tqdm(range(len(user_embeddings))):
        for i in range(items_number, len(item_embeddings)):
          score = list(user_embeddings[u:(u + 1)].dot(item_embeddings[i:(i+1)].T) + user_biases[:1].reshape(-1,1) + item_biases[i:(i+1)].reshape(1,-1))[0][0]
          pair = (i, score)
          pairs[u].append(pair)
          pairs[u] = sorted(pairs[u], key=lambda x: x[-1], reverse=True)
          pairs[u].remove(pairs[u][-1])

    model_dict = dict()
    for user, user_data in enumerate(pairs):
          for rank, (item, score) in enumerate(user_data):
              key = tuple([user, item])
              value = tuple([score, (rank + 1)])
              model_dict[key] = value

    model_pairs = list()
    for key in model_dict.keys():
        model_pairs.append(key)

    return model_pairs, model_dict

## HybridRecommender

Класс **HybridRecommender** реализует модель 2 этапа, ответственную за ранжирование товаров-кандидатов.

Метод prepare_data преобразовывает данные к требуемому формату, который ожидает на вход бустинг.

Метод train_test разделяет выборку на обучающую и тестовую части.

Метод fit обучает модель градиентного бустинга.

Метод recommend составляет рекомендации на основе обученной модели бустинга.

In [ ]:
class HybridRecommender:

  def __init__(self):
    pass

  def prepare_data(self, model_first_pairs, model_second_pairs, model_first_dict, model_second_dict, model_first_name, model_second_name, test_global_csr, train, test):
    """
    Метод составляет датафрейм для последующей подачи на вход бустингу, который выполнит итоговое ранжирование.

    Параметры:
    ----------
    model_first_pairs: list
      Множество пар вида (товар, скор) для каждого пользователя из users от первой модели. Этот и последующие 3 параметра реализуются методом candidates_extractor.candidates_extraction().
    model_second_pairs: list
      Множество пар вида (товар, скор) для каждого пользователя из users от второй модели.
    model_first_dict: dict
      Словарь, ключами которого явлюятся id пользователей, соответствующими им значениями - id товаров-кандидатов от первой модели.
    model_second_dict: dict
      Словарь, ключами которого явлюятся id пользователей, соответствующими им значениями - id товаров-кандидатов от второй модели.
    model_first_name: str
      Название первой модели. Используется для присвоения имён колонок итогового датафрейма dataset. Для того же служит и следующий параметр.
    model_second_name: str
      Название второй модели.
    test_global_csr: csr-matrix
      Разрезженная csr-матрица, построенная по тестовому датафрейму.
    train: Dataframe
      pandas-датафрейм, использовавшийся для обучения моделей первого уровня в виде csr-матрицы.
    test: Dataframe
      pandas-датафрейм, использовавшийся для тестирования моделей первого уровня в виде csr-матрицы.

    На выход:
    ----------
    dataset: Dataframe
      pandas-датафрейм с исходными индексами пользователей и товаров, который будет использоваться для обучения и тестирования бустинга.
    """
    total_pairs = list(set(model_first_pairs).union(set(model_second_pairs)))
    del model_first_pairs, model_second_pairs

    data_all_pairs = [pair +
                      model_first_dict.get(pair, (np.nan, np.nan)) +
                      model_second_dict.get(pair, (np.nan, np.nan))  for pair in tqdm(total_pairs)]
    del model_first_dict, model_second_dict

    first_score = model_first_name + "_score"
    second_score = model_second_name + "_score"
    first_rank = model_first_name + "_rank"
    second_rank = model_second_name + "_rank"
    data_all_pairs_df = pd.DataFrame(data_all_pairs,
                                 columns=["user_id", "item_id", first_score, first_rank,
                                                                second_score, second_rank])
    del data_all_pairs

    for column in data_all_pairs_df.columns:
      if column.endswith("id"):
        data_all_pairs_df[column] = data_all_pairs_df[column].astype(np.int32)
      else:
        data_all_pairs_df[column] = data_all_pairs_df[column].astype(np.float32)

    purchases = list()
    for k in range(test_global_csr.shape[0]):
      cx = coo_matrix(test_global_csr[k])
      purchased_items, user_id = [], []
      user_id.append(k)
      for i,j,v in zip(cx.row, cx.col, cx.data):
        purchased_items.append(j)
      for i in list(itertools.product(user_id, purchased_items)):
        purchases.append(i)
    del test_global_csr

    data_true = {}
    for i in purchases:
      curr, item = i[0], int(i[1])
      if curr not in data_true:
        data_true[curr] = list()
        data_true[curr].append(item)
      else:
        data_true[curr].append(item)
    for i in tqdm(data_true.keys()):
      data_true[i] = set(data_true[i])
    del purchases

    items_dict = dict(zip(train.item_new_id, train.item_id))
    users_dict = dict(zip(train.user_new_id, train.user_id))
    del train

    data_all_pairs_df["user_id"] = data_all_pairs_df["user_id"].map(users_dict)
    data_all_pairs_df["item_id"] = data_all_pairs_df["item_id"].map(items_dict)
    del items_dict, users_dict

    test["target"] = 1
    dataset = pd.merge(data_all_pairs_df,
                       test[["user_id", "item_id", "target"]].drop_duplicates(),
                       how="left",
                       left_on=["user_id", "item_id"],
                       right_on=["user_id", "item_id"])

    dataset["target"].fillna(0, inplace=True)
    del test
    return dataset


  def train_test(self, data, train_size, random_state=42):
    """
    Метод делит выборку, подготовленную для бустинга, на обучающую и тестовую части.

    Параметры:
    ----------
    data: Dataframe
      pandas-датафрейм с фичами (ранги и скоры), полученными из моделей отбора кандидатов, и бинаризованным таргетом, обозначающим, произошло ли взаимодействие пользователя с товаром.
    train_size: float
      Размер обучающей выборки, на которой будет обучаться градиентный бустинг.

    На выход:
    ----------
    X_train: Dataframe
      pandas-датафрейм из фичей для обучения градиентного бустинга.
    y_train: Series
      pandas-ряд, содержащий значения бинаризованного таргета, для обучения градиентного бустинга.
    X_test: Dataframe
      pandas-датафрейм из фичей для тестирования градиентного бустинга.
    y_test: Series
      pandas-ряд, содержащий значения бинаризованного таргета, для тестирования градиентного бустинга.
    train_query: Series
      Правило для разбиение обучающей выборки по пользователям, необходимый для градиентного бустинга.
    test_query: Series
      Правило для разбиение тестовой выборки по пользователям, необходимый для градиентного бустинга.
    """

    train_xy, test_xy = train_test_split(data, train_size=0.7, random_state=42)

    y_train = train_xy.pop("target")
    x_train = train_xy.copy()

    y_test = test_xy.pop("target")
    x_test = test_xy.copy()

    X_train = x_train[["warp_score", "warp_rank", "lmf_score", "lmf_rank"]]
    X_test = x_test[["warp_score", "warp_rank", "lmf_score", "lmf_rank"]]

    x_train = x_train.sort_values("user_id").reset_index(drop=True)
    x_test = x_test.sort_values("user_id").reset_index(drop=True)

    train_query = x_train["user_id"].value_counts().sort_index()
    test_query = x_test["user_id"].value_counts().sort_index()

    return X_train, y_train, X_test, y_test, train_query, test_query


  def fit(self, parameters, X_train, y_train, X_test, y_test, train_query, test_query, K_items=[20]):
    """
    Метод обучает градиентный бустинг на полученных ранее фичах от моделей первого уровня.

    Параметры:
    ----------
    parameters: dict
      Набор значений параметров для градиентного бустинга.
    X_train: Dataframe
      pandas-датафрейм из фичей для обучения градиентного бустинга.
    y_train: Series
      pandas-ряд, содержащий значения бинаризованного таргета, для обучения градиентного бустинга.
    X_test: Dataframe
      pandas-датафрейм из фичей для тестирования градиентного бустинга.
    y_test: Series
      pandas-ряд, содержащий значения бинаризованного таргета, для тестирования градиентного бустинга.
    train_query: Series
      Правило для разбиение обучающей выборки по пользователям, необходимый для градиентного бустинга.
    test_query: Series
      Правило для разбиение тестовой выборки по пользователям, необходимый для градиентного бустинга.
    K_items: list
      Сколько айтемов учитывать при вычислении метрики качества. По умолчанию равно [20].

    На выход:
    ----------
    model_gbm: LGBMRanker
      Обученная ранжирующая модель градиентного бустинга.
    """
    model_gbm = lightgbm.LGBMRanker(n_estimators=parameters["n_estimators"],
                                    objective=parameters["objective"],
                                    random_state=42)
    model_gbm.fit(X_train,
                  y_train,
                  group=train_query,
                  eval_set=[(X_test, y_test)],
                  eval_group=[list(test_query)],
                  eval_at=K_items)

    return model_gbm

  def recommend(self, model_gbm, x_test, K_items=20):
    """
    Мотод построит рекомендации по K_items товаров для каждого пользователя.

    Параметры:
    ----------
    model_gbm: LGBMRanker
      Обученная ранжирующая модель градиентного бустинга.
    x_test: Dataframe
      pandas-датафрейм, состоящий из пользователей, для которых необходимо сделать прогноз, и фичей товаров (ранги и скоры), полученные по моделям перовго уровня.
    K_items: int
      Сколько товаров необходимо порекомендовать каждому пользователю из датафрейма x_test. По умолчанию равно 20.

    На выход:
    dataset_predicted: dict
      Словарь, ключами которого являются user_id пользователей из x_test, а значениями - рекомендованные им товары.
    """

    lgb_test = x_test.copy()
    # lgb_test[["user_id", "item_id"]].drop_duplicates(inplace=True)
    lgb_test.set_index(["user_id", "item_id"], inplace=True)
    lgb_test["lgb_score"] = model_gbm.predict(lgb_test)
    # lgb_test = lgb_test.set_index("lgb_score", append=True).sort_values("lgb_score", ascending=False)
    # lgb_test.drop_duplicates(inplace=True)

    dataset_predicted = dict()
    lgb_test.reset_index(inplace=True)
    for user, group in tqdm(lgb_test.groupby("user_id")):
        dataset_predicted[user] = list(group.sort_values(by="lgb_score", ascending=False).item_id)[:K_items]
    return dataset_predicted

# Решение

С помощью созданных классов построим решение задачи. Первая реализация использует предпосчитанные скоры. Полный вариант немного ниже.

In [ ]:
# ###
# Подготовка данных для обучения моделей отбора кандидатов
data_preparator = DataPreparation()

df = data_preparator.extract_reluctant_users(data=df,
                                             threshold=20,
                                             both=False)

df = data_preparator.drop_rare_items(data=df,
                                      threshold=10)

train_global, test_global = data_preparator.train_test(data=df,
                                                        by="time",
                                                        test_weeks=1)
train_global, test_global = data_preparator.common_only(data_first=train_global,
                                                        data_second=test_global,
                                                        both=False,
                                                        common_col="user_id")

train_global = data_preparator.extract_reluctant_users(train_global)
train_global = data_preparator.drop_rare_items(data=train_global,
                                               threshold=20)
train_global, test_global = data_preparator.common_only(data_first=train_global,
                                                        data_second=test_global,
                                                        both=True)

train_global_csr, test_global_csr, train, test, users, items = data_preparator.csr_matrix_via_encoder(data_first=train_global,
                                                                                                      data_second=test_global)


# Скоры вычислены заранее, поэтому заново обучтаь модели не требуется.


###
# Извлечение кандидатов. Скоры вычислены заранее. Полный вариант представлен разделом ниже.

candidates_extractor = CandidatesExtractor()

warp_pairs, warp_dict = candidates_extractor.candidates_extraction(model_type="warp",
                                                                   path="/content/drive/MyDrive/WB School/L4/",
                                                                   prefix="global",
                                                                   users=users,
                                                                   items=items,
                                                                   top=50,
                                                                   items_number=50,
                                                                   precomputed_scores=True) # 2.94 GB

lmf_pairs, lmf_dict = candidates_extractor.candidates_extraction(model_type="lmf",
                                                                   path="/content/drive/MyDrive/WB School/L4/",
                                                                   prefix="global",
                                                                   users=users,
                                                                   items=items,
                                                                   top=50,
                                                                   items_number=50,
                                                                   precomputed_scores=True) # 2.52 GB

del df, users, items, train_global, test_global


###
# Бустинг над кандидатами
hybrid_recommender = HybridRecommender()
dataset = hybrid_recommender.prepare_data(model_first_pairs=warp_pairs,
                                          model_second_pairs=lmf_pairs,
                                          model_first_dict=warp_dict,
                                          model_second_dict=lmf_dict,
                                          model_first_name="warp",
                                          model_second_name="lmf",
                                          test_global_csr=test_global_csr,
                                          train=train,
                                          test=test) # 0.97 GB
del warp_pairs, lmf_pairs, warp_dict, lmf_dict, test_global_csr, train, test # -0.62 GB

X_train, y_train, X_test, y_test, train_query, test_query = hybrid_recommender.train_test(data=dataset, train_size=0.7, random_state=42) # 1.09 GB

parameters = {"n_estimators": 200,
              "objective": "lambdarank"}
model_gbm = hybrid_recommender.fit(parameters, X_train, y_train, X_test, y_test, train_query, test_query, K_items=[20])

Полная реализация алгоритма.

In [ ]:
# ###
# # Подготовка данных для обучения моделей отбора кандидатов
data_preparator = DataPreparation()

df = data_preparator.extract_reluctant_users(data=df,
                                             threshold=20,
                                             both=False)

df = data_preparator.drop_rare_items(data=df,
                                      threshold=10)

train_global, test_global = data_preparator.train_test(data=df,
                                                        by="time",
                                                        test_weeks=1)
train_global, test_global = data_preparator.common_only(data_first=train_global,
                                                        data_second=test_global,
                                                        both=False,
                                                        common_col="user_id")

train_global = data_preparator.extract_reluctant_users(train_global)
train_global = data_preparator.drop_rare_items(data=train_global,
                                               threshold=20)
train_global, test_global = data_preparator.common_only(data_first=train_global,
                                                        data_second=test_global,
                                                        both=True)

train_global_csr, test_global_csr, train, test, users, items = data_preparator.csr_matrix_via_encoder(data_first=train_global,
                                                                                                      data_second=test_global)

# ###
# # Обучение моделей отбора кандидатов
train_lightfm = TrainLightFM()

parameters_warp = {"no_components": 40,
                   "learning_schedule": "adagrad",
                   "learning_rate": 0.04,
                   "item_alpha": 0.00001,
                   "user_alpha": 0.0001,
                   "max_sampled": 40,
                   "epochs": 20}
model_warp = train_lightfm.fit(train_data_csr=train_global_csr,
                               loss="warp",
                               parameters=parameters_warp,
                               random_state=42)

parameters_lmf = {"no_components": 11,
                  "learning_schedule": "adagrad",
                  "learning_rate": 0.019,
                   "item_alpha": 0.00023,
                   "user_alpha": 0.00017,
                   "epochs": 20}
model_lmf = train_lightfm.fit(train_data_csr=train_global_csr,
                               loss="logistic",
                               parameters=parameters_lmf,
                               random_state=42)

###
# Извлечение кандидатов. Скоры вычислены заранее. Полный вариант представлен разделом ниже.

candidates_extractor = CandidatesExtractor()

warp_pairs, warp_dict = candidates_extractor.candidates_extraction(users=users,
                                                                   items=items,
                                                                   model=model_warp,
                                                                   top=50,
                                                                   items_number=50,
                                                                   precomputed_scores=False)

lmf_pairs, lmf_dict = candidates_extractor.candidates_extractor.candidates_extraction(users=users,
                                                                                      items=items,
                                                                                      model=model_warp,
                                                                                      top=50,
                                                                                      items_number=50,
                                                                                      precomputed_scores=False)

del df, users, items, train_global, test_global

###
# Бустинг над кандидатами
hybrid_recommender = HybridRecommender()
dataset = hybrid_recommender.prepare_data(model_first_pairs=warp_pairs,
                                          model_second_pairs=lmf_pairs,
                                          model_first_dict=warp_dict,
                                          model_second_dict=lmf_dict,
                                          model_first_name="warp",
                                          model_second_name="lmf",
                                          test_global_csr=test_global_csr,
                                          train=train,
                                          test=test) # 0.97 GB
del warp_pairs, lmf_pairs, warp_dict, lmf_dict, test_global_csr, train, test # -0.62 GB

X_train, y_train, X_test, y_test, train_query, test_query = hybrid_recommender.train_test(data=dataset, train_size=0.7, random_state=42) # 1.09 GB

parameters = {"n_estimators": 200,
              "objective": "lambdarank"}
model_gbm = hybrid_recommender.fit(parameters, X_train, y_train, X_test, y_test, train_query, test_query, K_items=[20])